<a href="https://colab.research.google.com/github/uroojfatima18/AI-Agents/blob/main/streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U q openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
from agents import Agent,Runner,AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled
from google.colab import userdata
from openai.types.responses import ResponseTextDeltaEvent

import nest_asyncio
nest_asyncio.apply()

In [ ]:
gemini_api_key= userdata.get('GEMINI_API_KEY')

In [ ]:
external_client=AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

Model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client,
)

set_tracing_disabled(disabled=True)

##Raw response event


In [ ]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner,AsyncOpenAI

async def main():
  agent=Agent(
      name="HELPER",
      instructions= "you are a helping assistant to help in user query",
      model=Model
  )
  result=Runner.run_streamed(agent , input = "tell me  3joke")

  async for event in result.stream_events():
     if  event.type =="raw_resonse_event"  and isinstance(event.data ,ResponseTextDeltaEvent):
      print(event.data.delta , end ='')

  print(result.final_output)
asyncio.run(main())

Okay, here are 3 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  What do you call a lazy kangaroo?
    Pouch potato!

3.  Why did the scarecrow win an award?
    Because he was outstanding in his field!



#Run item events and agent events

In [ ]:
import random
import asyncio
# from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner , ItemHelpers ,function_tool

@function_tool
def how_many_jokes():
  return random.randint(1,10)

async def main():
  agent=Agent(
      name="joker",
      instructions= "First call the `how_many_jokes` tool, then tell that many jokes.",
      model=Model,
      tools=[how_many_jokes]
  )
  result=Runner.run_streamed(agent , input = "hellow")

  async for event  in result.stream_events():
    if event.type=="raw_reponse_event":
      continue

    elif event.type =="agent_updated_streamed_event":
      print(f"Agent updated: {event.new_agent.name}")
      continue

    elif event.type =="run_item_streamed_event":
      if event.item.type=="tool_call_item":
         print(f"tool was called")

      elif event.type=="tool_call_output_item":
        print(f"tool called:{event.item.output}")

      elif event.type == "message_output_item":
        print(f"message output :\n{ItemHelpers.text_message_output(event.item)}")
      else:
          pass

asyncio.run(main())

